In [107]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

from IPython.display import Image
import graphviz
import pydotplus 

pd.options.display.max_columns = 25

In [54]:
df_austin = pd.read_csv("austin_weather.csv")

In [283]:
df['Binary'] = np.where(df.Events == 'clear', 1, 0)

C:\Users\RTFE4\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [284]:
df

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,SeaLevelPressureHighInches,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events,Month,Day,Binary
0,2013-12-21,74,60,45,67,49,43,93,75,57,29.86,29.68,29.59,10,7,2.0,20,4,31,0.46,"Rain , Thunderstorm",12,21,0
1,2013-12-22,56,48,39,43,36,28,93,68,43,30.41,30.13,29.87,10,10,5.0,16,6,25,0.00,clear,12,22,1
2,2013-12-23,58,45,32,31,27,23,76,52,27,30.56,30.49,30.41,10,10,10.0,8,3,12,0.00,clear,12,23,1
3,2013-12-24,61,46,31,36,28,21,89,56,22,30.56,30.45,30.3,10,10,7.0,12,4,20,0.00,clear,12,24,1
4,2013-12-25,58,50,41,44,40,36,86,71,56,30.41,30.33,30.27,10,10,7.0,10,2,16,0.00,clear,12,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,2017-07-27,103,89,75,71,67,61,82,54,25,30.04,29.97,29.88,10,10,10.0,12,5,21,0.00,clear,7,27,1
1315,2017-07-28,105,91,76,71,64,55,87,54,20,29.97,29.9,29.81,10,10,10.0,14,5,20,0.00,clear,7,28,1
1316,2017-07-29,107,92,77,72,64,55,82,51,19,29.91,29.86,29.79,10,10,10.0,12,4,17,0.00,clear,7,29,1
1317,2017-07-30,106,93,79,70,68,63,69,48,27,29.96,29.91,29.87,10,10,10.0,13,4,20,0.00,clear,7,30,1


In [281]:
df_austin.dtypes

Date                           object
TempHighF                       int64
TempAvgF                        int64
TempLowF                        int64
DewPointHighF                  object
DewPointAvgF                   object
DewPointLowF                   object
HumidityHighPercent            object
HumidityAvgPercent             object
HumidityLowPercent             object
SeaLevelPressureHighInches     object
SeaLevelPressureAvgInches      object
SeaLevelPressureLowInches      object
VisibilityHighMiles            object
VisibilityAvgMiles             object
VisibilityLowMiles            float64
WindHighMPH                    object
WindAvgMPH                     object
WindGustMPH                    object
PrecipitationSumInches        float64
Events                         object
Month                           int64
Day                             int64
dtype: object

In [56]:
df_austin['Events'].replace(' ', np.nan, inplace=True)
df_austin['Events'].fillna('clear', inplace=True)

In [57]:
# df_austin.replace('-', np.mean, inplace=True)

In [58]:
df_austin['PrecipitationSumInches'].replace('T', 0, inplace=True)

In [59]:
df_austin.head()

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,SeaLevelPressureHighInches,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,29.86,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,30.41,30.13,29.87,10,10,5,16,6,25,0,clear
2,2013-12-23,58,45,32,31,27,23,76,52,27,30.56,30.49,30.41,10,10,10,8,3,12,0,clear
3,2013-12-24,61,46,31,36,28,21,89,56,22,30.56,30.45,30.3,10,10,7,12,4,20,0,clear
4,2013-12-25,58,50,41,44,40,36,86,71,56,30.41,30.33,30.27,10,10,7,10,2,16,0,clear


In [184]:
no_use = ["Date", "Events"]
target = "Events"
features2 = [i for i in df_austin if i not in no_use]

In [185]:
features2

['TempHighF',
 'TempAvgF',
 'TempLowF',
 'DewPointHighF',
 'DewPointAvgF',
 'DewPointLowF',
 'HumidityHighPercent',
 'HumidityAvgPercent',
 'HumidityLowPercent',
 'SeaLevelPressureHighInches',
 'SeaLevelPressureAvgInches',
 'SeaLevelPressureLowInches',
 'VisibilityHighMiles',
 'VisibilityAvgMiles',
 'VisibilityLowMiles',
 'WindHighMPH',
 'WindAvgMPH',
 'WindGustMPH',
 'PrecipitationSumInches',
 'Month',
 'Day']

In [61]:
df_austin['Month'] = pd.to_datetime(df_austin.Date).dt.month
df_austin['Day'] =  pd.to_datetime(df_austin.Date).dt.day

In [62]:
df_austin.Date

0       2013-12-21
1       2013-12-22
2       2013-12-23
3       2013-12-24
4       2013-12-25
           ...    
1314    2017-07-27
1315    2017-07-28
1316    2017-07-29
1317    2017-07-30
1318    2017-07-31
Name: Date, Length: 1319, dtype: object

In [63]:
no_use = ["Events"]
target = "Events"
features2 = [i for i in df_austin if i not in no_use]
features = ['TempHighF', 'VisibilityLowMiles', 'PrecipitationSumInches', 'Date']

In [64]:
df_austin.dtypes

Date                          object
TempHighF                      int64
TempAvgF                       int64
TempLowF                       int64
DewPointHighF                 object
DewPointAvgF                  object
DewPointLowF                  object
HumidityHighPercent           object
HumidityAvgPercent            object
HumidityLowPercent            object
SeaLevelPressureHighInches    object
SeaLevelPressureAvgInches     object
SeaLevelPressureLowInches     object
VisibilityHighMiles           object
VisibilityAvgMiles            object
VisibilityLowMiles            object
WindHighMPH                   object
WindAvgMPH                    object
WindGustMPH                   object
PrecipitationSumInches        object
Events                        object
Month                          int64
Day                            int64
dtype: object

In [65]:
df_austin.DewPointAvgF.unique()

array(['49', '36', '27', '28', '40', '39', '41', '26', '42', '22', '48',
       '32', '8', '11', '45', '55', '61', '37', '47', '25', '23', '20',
       '33', '30', '29', '17', '14', '13', '54', '59', '15', '24', '34',
       '35', '57', '50', '53', '60', '46', '56', '51', '31', '38', '62',
       '43', '63', '64', '67', '66', '58', '70', '68', '65', '69', '71',
       '72', '-', '73', '74', '21', '44', '52', '12', '75', '76', '18'],
      dtype=object)

In [66]:
# df_austin['TempHighF'].mean()

In [67]:
df_austin.replace('-', np.nan, inplace=True)

In [68]:
'TempHighF', 'VisibilityLowMiles'

('TempHighF', 'VisibilityLowMiles')

In [69]:
df_austin['TempHighF'].uniqueque()

AttributeError: 'Series' object has no attribute 'uniqueque'

In [71]:
A = [int(i) for i in ['2', '5', '10', '7', '4', '8', '9', '1', '0', '3', '6']]

In [72]:
np.mean(A)

5.0

In [73]:
df_austin['VisibilityLowMiles'].unique()

array(['2', '5', '10', '7', '4', '8', '9', '1', '0', '3', '6', nan],
      dtype=object)

In [74]:
df_austin.fillna(0, inplace=True)

In [75]:
# df_austin.fillna(df_austin.mean().to_dict(), inplace=True)

In [76]:
df_austin.dtypes

Date                          object
TempHighF                      int64
TempAvgF                       int64
TempLowF                       int64
DewPointHighF                 object
DewPointAvgF                  object
DewPointLowF                  object
HumidityHighPercent           object
HumidityAvgPercent            object
HumidityLowPercent            object
SeaLevelPressureHighInches    object
SeaLevelPressureAvgInches     object
SeaLevelPressureLowInches     object
VisibilityHighMiles           object
VisibilityAvgMiles            object
VisibilityLowMiles            object
WindHighMPH                   object
WindAvgMPH                    object
WindGustMPH                   object
PrecipitationSumInches        object
Events                        object
Month                          int64
Day                            int64
dtype: object

In [77]:
features2

['Date',
 'TempHighF',
 'TempAvgF',
 'TempLowF',
 'DewPointHighF',
 'DewPointAvgF',
 'DewPointLowF',
 'HumidityHighPercent',
 'HumidityAvgPercent',
 'HumidityLowPercent',
 'SeaLevelPressureHighInches',
 'SeaLevelPressureAvgInches',
 'SeaLevelPressureLowInches',
 'VisibilityHighMiles',
 'VisibilityAvgMiles',
 'VisibilityLowMiles',
 'WindHighMPH',
 'WindAvgMPH',
 'WindGustMPH',
 'PrecipitationSumInches',
 'Month',
 'Day']

In [78]:
# d = pd.get_dummies(df_austin.Month)

In [79]:
# df_austin = pd.get_dummies(df_austin, columns=['Month'])

In [80]:
for i in df_austin:
    print(i)

Date
TempHighF
TempAvgF
TempLowF
DewPointHighF
DewPointAvgF
DewPointLowF
HumidityHighPercent
HumidityAvgPercent
HumidityLowPercent
SeaLevelPressureHighInches
SeaLevelPressureAvgInches
SeaLevelPressureLowInches
VisibilityHighMiles
VisibilityAvgMiles
VisibilityLowMiles
WindHighMPH
WindAvgMPH
WindGustMPH
PrecipitationSumInches
Events
Month
Day


In [81]:
# features = ['TempHighF', 'VisibilityLowMiles', 'PrecipitationSumInches', 'Month']
features = ['TempHighF', 'VisibilityLowMiles', 'PrecipitationSumInches', 'Month']

In [82]:
df_austin.Date.unique()

array(['2013-12-21', '2013-12-22', '2013-12-23', ..., '2017-07-29',
       '2017-07-30', '2017-07-31'], dtype=object)

In [83]:
# tree.DecisionTreeClassifier?

In [84]:
d

NameError: name 'd' is not defined

In [85]:
df_austin

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,SeaLevelPressureHighInches,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events,Month,Day
0,2013-12-21,74,60,45,67,49,43,93,75,57,29.86,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm",12,21
1,2013-12-22,56,48,39,43,36,28,93,68,43,30.41,30.13,29.87,10,10,5,16,6,25,0,clear,12,22
2,2013-12-23,58,45,32,31,27,23,76,52,27,30.56,30.49,30.41,10,10,10,8,3,12,0,clear,12,23
3,2013-12-24,61,46,31,36,28,21,89,56,22,30.56,30.45,30.3,10,10,7,12,4,20,0,clear,12,24
4,2013-12-25,58,50,41,44,40,36,86,71,56,30.41,30.33,30.27,10,10,7,10,2,16,0,clear,12,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,2017-07-27,103,89,75,71,67,61,82,54,25,30.04,29.97,29.88,10,10,10,12,5,21,0,clear,7,27
1315,2017-07-28,105,91,76,71,64,55,87,54,20,29.97,29.9,29.81,10,10,10,14,5,20,0,clear,7,28
1316,2017-07-29,107,92,77,72,64,55,82,51,19,29.91,29.86,29.79,10,10,10,12,4,17,0,clear,7,29
1317,2017-07-30,106,93,79,70,68,63,69,48,27,29.96,29.91,29.87,10,10,10,13,4,20,0,clear,7,30


In [99]:
df_austin['PrecipitationSumInches'] = df_austin['PrecipitationSumInches'].astype('float')
df_austin['VisibilityLowMiles'] = df_austin['VisibilityLowMiles'].astype('float')

In [100]:
features

['TempHighF', 'VisibilityLowMiles', 'PrecipitationSumInches', 'Month']

In [180]:
features2

['Date',
 'TempHighF',
 'TempAvgF',
 'TempLowF',
 'DewPointHighF',
 'DewPointAvgF',
 'DewPointLowF',
 'HumidityHighPercent',
 'HumidityAvgPercent',
 'HumidityLowPercent',
 'SeaLevelPressureHighInches',
 'SeaLevelPressureAvgInches',
 'SeaLevelPressureLowInches',
 'VisibilityHighMiles',
 'VisibilityAvgMiles',
 'VisibilityLowMiles',
 'WindHighMPH',
 'WindAvgMPH',
 'WindGustMPH',
 'PrecipitationSumInches',
 'Month',
 'Day']

In [187]:
for i in features:
    print(df_austin[i].unique())

[ 74  56  58  61  57  60  62  64  44  55  69  71  59  36  48  53  70  72
  75  67  76  82  63  40  39  49  65  80  46  32  38  86  81  83  77  52
  73  47  50  68  66  89  79  85  84  87  78  88  96  93  94  91  92  90
  95  97  98 100 101  99 102 104 103  51  41  45  54  42  43  37 105  34
 107 106]
[ 2.  5. 10.  7.  4.  8.  9.  1.  0.  3.  6.]
[0.46 0.   0.16 0.1  0.01 0.06 0.05 0.02 0.15 0.11 0.08 0.17 0.74 0.07
 0.2  0.27 1.34 2.45 0.94 0.14 0.19 1.56 1.75 0.55 1.49 0.24 0.49 0.31
 3.53 1.52 0.09 0.98 0.22 0.51 0.68 0.35 3.66 0.13 0.21 0.03 1.42 0.3
 0.56 1.51 0.04 0.33 3.33 0.59 0.63 0.76 2.07 0.4  0.26 0.45 2.17 1.17
 0.43 0.41 0.57 3.84 0.29 0.23 1.12 0.36 2.6  1.41 5.2  0.67 2.66 1.09
 0.93 0.75 1.05 0.79 4.79 0.65 4.93 0.89 0.53 1.03 1.46 0.25 1.54 1.32
 1.33 2.18 0.34 1.19 1.13 0.58 0.54 0.92 0.77 2.25 0.52 0.18 1.07 1.61
 1.06 2.35 1.79 1.22 1.29 0.37 0.61 1.57 0.86 0.66 0.73 0.71 0.12 0.47
 1.39]
[12  1  2  3  4  5  6  7  8  9 10 11]


In [353]:
features

['TempHighF', 'VisibilityLowMiles', 'PrecipitationSumInches', 'Month']

In [360]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df['Binary'], random_state=0) 
binary_tree_class = tree.DecisionTreeClassifier(
    random_state=0, max_depth=4,criterion='entropy',max_features=len(features)
)
tree_class.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(tree_class.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(tree_class.score(X_test, y_test)))

ValueError: max_features must be in (0, n_features]

In [346]:
cross = cross_val_score(binary_tree_class, df[features2], df['Binary'], cv=5)

In [347]:
print(cross)
print(np.mean(cross))

[0.90909091 0.96590909 0.90909091 0.94296578 0.92748092]
0.9309075209178248


In [314]:
bag_binary = BaggingClassifier(
    binary_tree_class, random_state=0, n_estimators=300
)
bag_binary.fit(X_train, y_train)
    # print("Правильность на обучающем наборе: {:.3f}".format(bag.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(bag_binary.score(X_test, y_test)))

Правильность на тестовом наборе: 0.900


In [123]:
df_austin[['Events', 'Date']].groupby(['Events']).count()

,Date
Events,
Fog,21
"Fog , Rain",14
"Fog , Rain , Thunderstorm",33
"Fog , Thunderstorm",1
Rain,192
"Rain , Snow",1
"Rain , Thunderstorm",137
Thunderstorm,17
clear,903


In [133]:
df[['Events', 'Date']].groupby(['Events']).count()

,Date
Events,
Fog,21
"Fog , Rain",14
"Fog , Rain , Thunderstorm",33
Rain,192
"Rain , Thunderstorm",137
Thunderstorm,17
clear,903


In [124]:
df = df_austin[(df_austin.Events != 'Fog , Thunderstorm') & (df_austin.Events != 'Rain , Snow')]

In [125]:
df.describe()

,TempHighF,TempAvgF,TempLowF,VisibilityLowMiles,PrecipitationSumInches,Month,Day
count,1317.000000,1317.000000,1317.000000,1317.000000,1317.000000,1317.000000,1317.000000
mean,80.875475,70.662870,59.930144,6.786636,0.114525,6.170843,15.798785
std,14.760128,14.034216,14.172182,3.719562,0.428486,3.415119,8.820986
min,32.000000,29.000000,19.000000,0.000000,0.000000,1.000000,1.000000
25%,72.000000,62.000000,49.000000,3.000000,0.000000,3.000000,8.000000
50%,83.000000,73.000000,63.000000,9.000000,0.000000,6.000000,16.000000
75%,92.000000,83.000000,73.000000,10.000000,0.000000,9.000000,23.000000
max,107.000000,93.000000,81.000000,10.000000,5.200000,12.000000,31.000000


In [269]:
cross_val_score(tree_class, df[features2], df[target], cv=5)

array([0.85393258, 0.85338346, 0.86311787, 0.83908046, 0.83846154])

In [191]:
for i, j in zip(features2, tree_class.feature_importances_):
    print(i, j)

TempHighF 0.04734454386195482
TempAvgF 0.0
TempLowF 0.04040992124300681
DewPointHighF 0.0054425036647735205
DewPointAvgF 0.0
DewPointLowF 0.0
HumidityHighPercent 0.0
HumidityAvgPercent 0.005004615932201579
HumidityLowPercent 0.029668737210415234
SeaLevelPressureHighInches 0.0
SeaLevelPressureAvgInches 0.012475424647474772
SeaLevelPressureLowInches 0.012567100518491792
VisibilityHighMiles 0.0
VisibilityAvgMiles 0.010087036395787737
VisibilityLowMiles 0.21281885552541013
WindHighMPH 0.015294112484609218
WindAvgMPH 0.0
WindGustMPH 0.014919356890167388
PrecipitationSumInches 0.5875895741347801
Month 0.006378217490926711
Day 0.0


In [209]:
for i in range(20, 100, 10):
    bag = BaggingClassifier(
        tree_class, random_state=0, n_estimators=i
    )
    bag.fit(X_train, y_train)
    # print("Правильность на обучающем наборе: {:.3f}".format(bag.score(X_train, y_train)))
    print("Правильность на тестовом наборе: {:.3f}".format(bag.score(X_test, y_test)))

Правильность на тестовом наборе: 0.839
Правильность на тестовом наборе: 0.839
Правильность на тестовом наборе: 0.839
Правильность на тестовом наборе: 0.842
Правильность на тестовом наборе: 0.845
Правильность на тестовом наборе: 0.848
Правильность на тестовом наборе: 0.842
Правильность на тестовом наборе: 0.845


In [612]:
df_austin.median()

TempHighF    83.0
TempAvgF     73.0
TempLowF     63.0
Month         6.0
Day          16.0
dtype: float64

In [631]:
BaggingClassifier?

In [317]:
dot_data = tree.export_graphviz(decision_tree=tree_class, out_file="tree.out",
                                max_depth=7,
                                feature_names=features2,
                                class_names=df_austin[target].unique(),
                                filled=True, rounded=True,  
                                special_characters=True,
                                proportion = False, precision=2,
                                label='all', impurity=False,
)  

graph = pydotplus.graphviz.graph_from_dot_file("tree.out")
Image(graph.create_png())

ValueError: Length of feature_names, 20 does not match number of features, 4

In [234]:
X_train, X_test, y_train, y_test = train_test_split(df[features2], df[target], random_state=42) 
forest = RandomForestClassifier(random_state=42, n_estimators=500, max_depth=10)
forest.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(forest.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(forest.score(X_test, y_test)))

Правильность на обучающем наборе: 0.986
Правильность на тестовом наборе: 0.852


In [265]:
X_train, X_test, y_train, y_test = train_test_split(df[features2], df[target], random_state=34) 
tree_class = tree.DecisionTreeClassifier(
    random_state=42, max_depth=8, criterion='entropy', max_features=int(len(features))
)
bag = BaggingClassifier(tree_class, random_state=0, n_estimators=80)
bag.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(bag.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(bag.score(X_test, y_test)))

Правильность на обучающем наборе: 0.964
Правильность на тестовом наборе: 0.894


In [266]:
cross_val_score(bag, df[features2], df[target], cv=5)

array([0.83895131, 0.88345865, 0.87072243, 0.82375479, 0.85769231])

In [329]:
X_train, X_test, y_train, y_test = train_test_split(df_austin[features2], df_austin[target], random_state=34) 
tree_class = tree.DecisionTreeClassifier(
    random_state=0, max_depth=4, criterion='entropy', max_features=len(features),
)
tree_class.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(tree_class.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(tree_class.score(X_test, y_test)))

Правильность на обучающем наборе: 0.839
Правильность на тестовом наборе: 0.836


In [330]:
for i in range(1, len(features)):
    X_train, X_test, y_train, y_test = train_test_split(df_austin[features], df_austin[target], random_state=42) 
    tree_class = tree.DecisionTreeClassifier(random_state=0, max_depth=5, criterion='entropy', max_features=i)
    tree_class.fit(X_train, y_train)
    # print("Правильность на обучающем наборе: {:.3f}".format(tree_class.score(X_train, y_train)))
    print("Правильность на тестовом наборе: {:.3f}".format(tree_class.score(X_test, y_test)))

Правильность на тестовом наборе: 0.836
Правильность на тестовом наборе: 0.809
Правильность на тестовом наборе: 0.845


Правильность на обучающем наборе: 0.846
Правильность на тестовом наборе: 0.773


In [372]:
tree.DecisionTreeClassifier?

In [ ]:
BaggingClassifier?